In [222]:

from sklearn.metrics import *
from sklearn.model_selection import KFold , cross_val_score, cross_validate, GridSearchCV
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import time
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

In [223]:
import os
import mglearn
import pandas as pd
adult = pd.read_csv(
    os.path.join(mglearn.datasets.DATA_PATH, "adult.data"), header=None, index_col=False,
    names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'gender',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
           'income'])

adult.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [224]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   gender          32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [225]:
#범주 데이터
print(adult.gender.value_counts())
print(adult.workclass.value_counts())
print(adult.race.value_counts())
print(adult.occupation.value_counts())

 Male      21790
 Female    10771
Name: gender, dtype: int64
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64
 White                 27816
 Black                  3124
 Asian-Pac-Islander     1039
 Amer-Indian-Eskimo      311
 Other                   271
Name: race, dtype: int64
 Prof-specialty       4140
 Craft-repair         4099
 Exec-managerial      4066
 Adm-clerical         3770
 Sales                3650
 Other-service        3295
 Machine-op-inspct    2002
 ?                    1843
 Transport-moving     1597
 Handlers-cleaners    1370
 Farming-fishing       994
 Tech-support          928
 Protective-serv       649
 Priv-house-serv       149
 Armed-Forces            9
Name: occupation, dtype: int64


In [226]:
adult.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [227]:
adult.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
gender            0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [228]:
#원 핫 인코딩 
adult_dummies = pd.get_dummies(adult)

In [229]:
print("get_dummies:\n", list(adult_dummies.columns))

get_dummies:
 ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Never-worked', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_ 10th', 'education_ 11th', 'education_ 12th', 'education_ 1st-4th', 'education_ 5th-6th', 'education_ 7th-8th', 'education_ 9th', 'education_ Assoc-acdm', 'education_ Assoc-voc', 'education_ Bachelors', 'education_ Doctorate', 'education_ HS-grad', 'education_ Masters', 'education_ Preschool', 'education_ Prof-school', 'education_ Some-college', 'marital-status_ Divorced', 'marital-status_ Married-AF-spouse', 'marital-status_ Married-civ-spouse', 'marital-status_ Married-spouse-absent', 'marital-status_ Never-married', 'marital-status_ Separated', 'marital-status_ Widowed', 'occupation_ ?', 'occupation_ Adm-clerical', 'occupation_ Armed-Forces', 'occupation_ 

In [230]:
adult_dummies.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,income_ <=50K,income_ >50K
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,53,234721,7,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,28,338409,13,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [231]:
features = adult_dummies.loc[:,'age':'native-country_ Yugoslavia']

In [232]:
# numpy
X = features.values
y = adult_dummies['income_ <=50K']

In [233]:
print("X.shape : {}  y.shape : {}".format(X.shape,y.shape))

X.shape : (32561, 108)  y.shape : (32561,)


*RandomForest*

In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(26048, 108) (6513, 108) (26048,) (6513,)


In [235]:
RF = RandomForestClassifier(n_estimators=100,max_depth=20,random_state=0)
RF.fit(X_train,y_train)

RandomForestClassifier(max_depth=20, random_state=0)

In [236]:
mypredictions = RF.predict(X_test)
print('RandomForest의 정확도는:{0:.4f}'.format(accuracy_score(y_test, mypredictions)))
print("RandomForest의 F1스코어:{0:.4f}".format(f1_score(y_test, mypredictions)))
print("RandomForest의 정밀도는:{0:.4f}".format(precision_score(y_test,mypredictions)))
print('RandomForest의 재현율은:{0:.4f}'.format(recall_score(y_test, mypredictions)))

RandomForest의 정확도는:0.8701
RandomForest의 F1스코어:0.9182
RandomForest의 정밀도는:0.8934
RandomForest의 재현율은:0.9443


*RandomForest의 정확도는:0.8701

*RandomForest의 F1스코어:0.9182

*RandomForest의 정밀도는:0.8934

*RandomForest의 재현율은:0.9443

In [214]:
#LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
LoRg = LogisticRegression(solver='liblinear')
LoRg.fit(X_train, y_train)
print("테스트 점수: {:.4f}".format(LoRg.score(X_test, y_test)))

테스트 점수: 0.8079


In [215]:
LoRg.fit(X_train,y_train)

mypredictions = LoRg.predict(X_test)

In [216]:
print('LoRg의 정확도는:{0:.4f}'.format(accuracy_score(y_test, mypredictions)))
print("LoRg의 F1스코어:{0:.4f}".format(f1_score(y_test, mypredictions)))
print("LoRg의 정밀도는:{0:.4f}".format(precision_score(y_test,mypredictions)))
print('LoRg의 재현율은:{0:.4f}'.format(recall_score(y_test, mypredictions)))

LoRg의 정확도는:0.8079
LoRg의 F1스코어:0.8864
LoRg의 정밀도는:0.8176
LoRg의 재현율은:0.9678


In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)

In [218]:
clf = svm.SVC(gamma=0.003,C=150)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


In [219]:
print('SVM의 정확도는:{0:.4f}'.format(accuracy_score(y_test, pred)))
print("SVM의 F1스코어:{0:.4f}".format(f1_score(y_test, pred)))
print("SVM의 정밀도는:{0:.4f}".format(precision_score(y_test,pred)))
print('SVM의 재현율은:{0:.4f}'.format(recall_score(y_test, pred)))

SVM의 정확도는:0.7294
SVM의 F1스코어:0.8374
SVM의 정밀도는:0.7673
SVM의 재현율은:0.9216


In [220]:
dT_clf = DecisionTreeClassifier(random_state = 30)
dT_clf.fit(X_train, y_train)
pred_dt = dT_clf.predict(X_test)


In [221]:
print('DecisionTree의 정확도는:{0:.4f}'.format(accuracy_score(y_test, pred_dt)))
print("DecisionTree의 F1스코어:{0:.4f}".format(f1_score(y_test, pred_dt)))
print("DecisionTree의 정밀도는:{0:.4f}".format(precision_score(y_test,pred_dt)))
print('DecisionTree의 재현율은:{0:.4f}'.format(recall_score(y_test, pred_dt)))

DecisionTree의 정확도는:0.8098
DecisionTree의 F1스코어:0.8737
DecisionTree의 정밀도는:0.8774
DecisionTree의 재현율은:0.8700


In [ ]:
params = {
    'max_depth':[2,4,6,8.10,12,16,20,24]
}
grid_cv = GridSearchCV(dT_clf, param_grid=params,scoring='accuracy',cv=5,verbose=1)
grid_cv.fit(X_train, y_train)
print('Grid 평균 정확도:{0:.f4}',format(grid_cv.best_score_))
print("Grid 최적 파라미터:", grid_cv.best_params_)